In [12]:
import pandas as pd
import boto3
from io import BytesIO
s3_cl = boto3.client('s3') # for lower-level processes

asosawos = s3_cl.get_object(Bucket='wecc-historical-wx', Key='1_raw_wx/ASOSAWOS/stationlist_ASOSAWOS.csv')
asosawos_list = pd.read_csv(BytesIO(asosawos['Body'].read()))  

asosawos_list # 399 records
    
isd = s3_cl.get_object(Bucket='wecc-historical-wx', Key='1_raw_wx/ASOSAWOS/stationlist_ISD_ASOSAWOS.csv')
isd_list = pd.read_csv(BytesIO(isd['Body'].read()))  

print(isd_list)

#print(asosawos_list.columns)
#print(isd_list.columns)

# Try match on WBAN, lat, lon
#print(asosawos_list[['WBAN', 'LAT', 'LON']])
#print(isd_list[['WBAN', 'LAT', 'LON']])

# Round asosawos down to 3 decimal points of accuracy
asosawos_round = asosawos_list.round({'LAT':3, 'LON':3})

# To do so, try exact match first
join = isd_list.merge(asosawos_round, left_on = ['WBAN', 'LAT', 'LON'], right_on = ['WBAN', 'LAT', 'LON'], how = 'inner')
join # 413 matches

missed = asosawos_list[~asosawos_list.NCDCID.isin(join.NCDCID)]
missed

    

,Unnamed: 0,NCDCID,WBAN,COOPID,CALL,NAME,ALTNAME,COUNTRY,ST,COUNTY,LAT,LON,ELEV,UTC,STNTYPE,STARTDATE,GHCN-DailyID,Barometer_elev,Anemometer_elev
0,0,10001445,93107,NaN,NKX,SAN DIEGO MIRAMAR NAS,SAN DIEGO MIRAMAR NAS,UNITED STATES,CA,SAN DIEGO,32.86667,-117.13333,145.3896,-8,"ASOS,MILITARY",19460501.0,USW00093107,-99999.0,NaN
1,1,20001424,23199,NaN,NJK,EL CENTRO NAF,EL CENTRO NAF,UNITED STATES,CA,IMPERIAL,32.83241,-115.66377,-14.0208,-8,ASOS,19900601.0,USW00023199,-99999.0,33.0
2,2,20001392,93115,NaN,NRS,IMPERIAL BEACH REAM FLD NAS,IMPERIAL BEACH REAM FLD NAS,UNITED STATES,CA,SAN DIEGO,32.56797,-117.11715,7.3152,-8,ASOS,19900601.0,USW00093115,20.0,33.0
3,3,20003403,23061,50130.0,ALS,ALAMOSA-BERGMAN FLD,ALAMOSA-BERGMAN FLD,UNITED STATES,CO,ALAMOSA,37.43931,-105.86180,2297.2776,-7,"ASOS,COOP,PLCD,WXSVC",19920901.0,USW00023061,7536.0,33.0
4,4,20003486,93058,56740.0,PUB,PUEBLO MEM AP,PUEBLO MEM AP,UNITED STATES,CO,PUEBLO,38.28868,-104.50570,1424.6352,-7,"ASOS,COOP,PLCD,WXSVC",19921001.0,USW00093058,4655.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,394,30003059,94086,NaN,PNA,PINEDALE WENZ FLD AP,NaN,UNITED STATES,WY,SUBLETTE,42.79556,-109.80694,-30479.6952,-7,AWOS,NaN,NaN,NaN,NaN
395,395,20022657,24027,487845.0,RKS,ROCK SPRINGS AP,NaN,UNITED STATES,WY,SWEETWATER,41.59465,-109.05290,2059.8384,-7,"AIRWAYS,ASOS,AWOS,COOP,USHCN,WXSVC",NaN,NaN,NaN,NaN
396,396,30083363,342,NaN,FWZ,S PASS,NaN,UNITED STATES,WY,FREMONT,42.51700,-108.78300,2588.0568,-7,AWOS,NaN,NaN,NaN,NaN
397,397,30083498,475,NaN,SAA,SHIVELY FIELD AP,NaN,UNITED STATES,WY,CARBON,41.44352,-106.82752,2138.4768,-7,AWOS,NaN,NaN,NaN,NaN
